In [69]:
import matplotlib.pyplot as plt
from matplotlib.colors import *
from matplotlib.patches import *
import numpy as np
import ipywidgets as widgets
import time
from sklearn.linear_model import *
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from scipy.optimize import *
from sklearn.ensemble import RandomForestRegressor


In [55]:
class VirtualSideKick():
    
     
    def __init__(self,colors=[]):
        
        
        if len(colors) == 0:
            colors = np.random.rand(5,3)
            self.color_count = 5
        else:
            self.color_count = len(colors)
            
        self.colors = np.array([rgb_to_hsv(x) for x in colors])
            
        self.intensity = [0.5] * self.color_count
        
        self.max_intensity = 1.5

        
    def setIntensity(self,i,f):
        self.intensity[i] = f

    def setIntensities(self,f):
        if len(f) != self.color_count:
            print("Formats do not match .. pick five intensities")
            return
        
        self.intensity = np.clip(f,0,1)
        
    def getRGB(self,intensities=None):
        
        if intensities is not None:
            #print("Int: ",intensities)
            self.setIntensities(intensities)
        
        color = np.zeros(3)
        for c,i in zip(self.colors,self.intensity):
            color += hsv_to_rgb(c * [1,1,i])
        
        color = np.clip(color,0,self.max_intensity) / self.max_intensity
    
        return color
        
class ModelState():
    
    def __init__(self,model,sidekick):
        
        self.model = model
        
        self.X = np.empty((0,sidekick.color_count))
        self.Y = np.empty((0,3))
        
        self.box = np.array([np.zeros(sidekick.color_count),np.ones(sidekick.color_count)],dtype=np.float32)
        
    def reset(self):
        
        self.X = np.empty((0,self.X.shape[1]))
        self.Y = np.empty((0,3))
        
        self.box = np.array([np.zeros(self.X.shape[1]),np.ones(self.X.shape[1])],dtype=np.float32)
        
        
sidekick = VirtualSideKick()


In [56]:
def setState(sidekick,button):
    rgb.value = np.array2string(sidekick.getRGB())
    rgb.send_state("value")
    
    button.style.button_color = to_hex(sidekick.getRGB())
    
def newTarget(target,button,state):
    color = np.random.rand(3)
    #color = np.array([0.07953693, 0.35833365, 0.55432933])
    target.value = np.array2string(color)
    target.send_state("value")

    button.style.button_color = to_hex(color)
    
    state.reset()

In [57]:
def solve_color(sidekick, target, count=10, corners=True):
    
    print("Solving with {} samples".format(count))
    X = np.random.rand(count,5)
    
    if corners:
        corners = np.zeros((32,5),dtype=np.float32)
        for i in range(0,32):
            corners[i,:] = np.array([i%2, (i//2)%2, (i//4)%2, (i//8)%2, (i//16)%2 ])
    
        X = np.concatenate((corners, X))

    y = [sidekick.getRGB(x) for x in X]
    
    reg = Ridge(0.8)
    #reg = MLPRegressor(hidden_layer_sizes=(3,5,5,3),activation='relu')

    reg.fit(y,X)

    return sidekick.getRGB(reg.predict(target.reshape(1, -1))[0])

    
def solve_color_callback(color_now, color_target, sidekick):
    
   
    target = np.array(to_rgb(color_target.style.button_color))
    
    best_guess = solve_color(sidekick,target)
    
    setState(sidekick,color_now)
    

In [66]:
def adaptive_solve_callback(sidekick, target, state, color_now, color_guess,
                            samples_per_round=10, candidate_count = 1000, window=20):
    
    # Define N as the number of inputs (number of outputs will always be 3)
    N = sidekick.color_count
    
    #print(state.Y)
    
    # If we have current samples we assume we have a model 
    if state.X.shape[0] > 0:
        
        # Which means we can guess the next window to sample
 
        # Create candidate samples in our current box
        candidates = np.random.rand(candidate_count,N)
        candidates = candidates * (state.box[1] - state.box[0]) + state.box[0]
    
        # Evaluate the candidates
        predict = state.model.predict(candidates)
        
        # Compute the current loss as L2**2 between the predictions and the target
        loss = np.sum((predict - target)**2,axis=-1)
        
        # And sort the candidates by ascending order
        candidates = candidates[loss.argsort()]

        # Define the new box as the bounding box of the 
        # window best samples 
        for i in range(0,N):
            state.box[0,i] = candidates[0:window,i].min()
            state.box[1,i] = candidates[0:window,i].max()

            
    
    # Create the next round of ground truth samples
    X = np.random.rand(samples_per_round,N)
    # In the current box
    X = X * (state.box[1] - state.box[0]) + state.box[0]

    Y = np.zeros((samples_per_round,3),np.float32)
    
    # Evaluate the samples and show the user what colors we are trying
    for i,x in enumerate(X):
        Y[i,:] = sidekick.getRGB(x)
        setState(sidekick,color_now)
        time.sleep(0.2)
        
    # Add these samples to the current state
    state.X = np.concatenate((state.X,X))
    state.Y = np.concatenate((state.Y,Y))
        
    # Fit the regression with the first set of samples
    state.model.fit(state.X,state.Y)

    # Once we are done with refitting the regressor we use 
    # an optimization approach to define our best guess. To 
    # find a good starting position we use yet more candidates
    # but in the current box 
    candidates = np.random.rand(candidate_count,N) # sidekick sliders
    candidates = candidates * (state.box[1] - state.box[0]) + state.box[0]

    predict = state.model.predict(candidates)
    loss = np.sum((predict - target)**2,axis=-1)
    candidates = candidates[loss.argsort()]
    
    # And we use the best candidate as starting point
    optimum = fmin(lambda x: np.inner(state.model.predict(x.reshape(-1,N))[0]-target,
                                      state.model.predict(x.reshape(-1,N))[0]-target),candidates[0])
    
    # Display the current best buess
    setState(sidekick,color_now)
    setState(sidekick,color_guess)

    return optimum

def adaptive_solve(sidekick, target, samples_per_round=10, rounds=2, candidate_count = 1000, window=20):
    
    # Define which regressor we will use
    #reg = Ridge(0.8)
    reg = RandomForestRegressor()
    
    # Define N as the number of inputs (number of outputs will always be 3)
    N = sidekick.color_count
    
    # Create the first round of ground truth samples
    X = np.random.rand(samples_per_round,N) 
    Y = np.array([sidekick.getRGB(x) for x in X])

    # Fit the regression with the first set of samples
    reg.fit(X,Y)

    # Initialize our sampling box 
    box = np.array([np.zeros(N),np.ones(N)],dtype=np.float32)
   
    # Make sure the candidates variable exists
    candidates = None
    
    # If we want to do any adaptive sampling
    for round in range(0,rounds-1):
 
        # Create more candidate samples in our current box
        candidates = np.random.rand(candidate_count,N)
        candidates = candidates * (box[1] - box[0]) + box[0]
    
        #for i in range(0,3):
        #    candidates[:,i] = box[0,i] + candidates[:,i]/(box[1,i] - box[0,i])

        # Evaluate the candidates
        predict = reg.predict(candidates)
        
        # Compute the current loss as L2**2 between the predictions and the target
        loss = np.sum((predict - target)**2,axis=-1)
        
        # And sort the candidates by ascending order
        candidates = candidates[loss.argsort()]

        # Define the new box as the bounding box of the 
        # window best samples 
        for i in range(0,N):
            box[0,i] = candidates[0:window,i].min()
            box[1,i] = candidates[0:window,i].max()
    
        # Create yet more random samples in this tighter box
        candidates = np.random.rand(samples_per_round,N)
        candidates = candidates * (box[1] - box[0]) + box[0]
        
        # Add these samples to the current inputs
        X = np.concatenate((X,candidates))
        
        # And actually evaluate them 
        Y = np.concatenate((Y,np.array([sidekick.getRGB(x) for x in candidates])))
            
        # Finally, refit the regressor from all samples
        reg.fit(X,Y)
   
    # Once we are done with refitting the regressor we use 
    # an optimization approach to define our best guess. To 
    # find a good starting position we use yet more candidates
    # but in the current box 
    candidates = np.random.rand(candidate_count,N) # sidekick sliders
    candidates = candidates * (box[1] - box[0]) + box[0]

    predict = reg.predict(candidates)
    loss = np.sum((predict - target)**2,axis=-1)
    candidates = candidates[loss.argsort()]
    
    # And we use the best candidate as starting point
    optimum = fmin(lambda x: np.inner(reg.predict(x.reshape(-1,N))[0]-target,
                                          reg.predict(x.reshape(-1,N))[0]-target),candidates[0])
#                      method='TNC',bounds=[(0,1)]*N)
    
    return optimum


# Test code that might make no sense
#sidekick = VirtualSideKick([[1,0,0],[0,1,0],[0,0,1],[0,0.5,0.5],[0.5,0.5,0]])
#target = sidekick.getRGB(np.random.rand(3))
#target = np.random.rand(3)

#optimum = adaptive_solve(sidekick,target,rounds=3,samples_per_round=10)

#print(target)
#print(optimum)
#print(sidekick.getRGB(optimum))

#print(sidekick.getRGB(candidates[0]))
#print(target)


#hsv = reg.predict(target.reshape(1, -1))
#rgb = hsv_to_rgb(hsv[0])

#print(target)
#print(truth)
#print(hsv)
#print(sidekick.getRGB(hsv[0]))
#print(rgb)



In [67]:
    
sidekick = VirtualSideKick([[1,0,0],[0,1,0],[0,0,1],[0.5,0.5,0],[0,0.5,0.5]])
state = ModelState(RandomForestRegressor(),sidekick)

#sidekick = VirtualSideKick()

sliders = [widgets.FloatSlider(value=0.5,min=0,max=1,step=0.005,description='',disabled=False,
                             continuous_update=True, orientation='horizontal') for i in range(0,5)]
rgb = widgets.Text(value=np.array2string(sidekick.getRGB()), description='Current RGB', layout=widgets.Layout(width='40%', disabled=False))


color = np.random.rand(3)
target = widgets.Text(value=np.array2string(color), description='target RGB', layout=widgets.Layout(width='40%', disabled=False))

button = widgets.Button(description="New Target",disabled=False,
                        layout=widgets.Layout(width='45%', height='30px'))
   
solve = widgets.Button(description="Solve",disabled=False,
                        layout=widgets.Layout(width='45%', height='30px'))

color_now = widgets.Button(description="Current",style=dict(button_color=to_hex(sidekick.getRGB())),
                           layout=widgets.Layout(width='300px', height='300px',disabled=False))
color_guess = widgets.Button(description="Best Guess",style=dict(button_color=to_hex(sidekick.getRGB())),
                            layout=widgets.Layout(width='300px', height='300px',disabled=False))
color_target = widgets.Button(description="Target",style=dict(button_color=to_hex(color)),
                            layout=widgets.Layout(width='300px', height='300px',disabled=False))
    
    
display(widgets.VBox([widgets.HBox(sliders),widgets.HBox([rgb,target]),
                      widgets.HBox([button,solve]),
                      widgets.HBox([color_now,color_guess, color_target])]))

button.on_click(lambda x: newTarget(target,color_target,state))


#solve.on_click(lambda x: solve_color_callback(color_now,color_target,sidekick))
solve.on_click(lambda x: adaptive_solve_callback(sidekick, color, state, color_now,color_guess,
                                                10,1000,50))

for i in range(0,5):
    sliders[i].observe(lambda x, arg=i: sidekick.setIntensity(arg,x['new']),"value")
    sliders[i].observe(lambda x: setState(sidekick,color_now))
    sliders[i].observe(lambda x: setState(sidekick,color_guess))
    


Optimization terminated successfully.
         Current function value: 0.002648
         Iterations: 10
         Function evaluations: 69


/Users/bremer5/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Optimization terminated successfully.
         Current function value: 0.000046
         Iterations: 11
         Function evaluations: 71
Optimization terminated successfully.
         Current function value: 0.000121
         Iterations: 11
         Function evaluations: 71
Optimization terminated successfully.
         Current function value: 0.000059
         Iterations: 12
         Function evaluations: 73
Optimization terminated successfully.
         Current function value: 0.000025
         Iterations: 17
         Function evaluations: 75
Optimization terminated successfully.
         Current function value: 0.000051
         Iterations: 12
         Function evaluations: 67
Optimization terminated successfully.
         Current function value: 0.000025
         Iterations: 17
         Function evaluations: 74
Optimization terminated successfully.
         Current function value: 0.000013
         Iterations: 13
         Function evaluations: 74
